# Cargar librerías y datsets

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk 
import seaborn as sns
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import make_pipeline
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
train_data = pd.read_json("train_data_LIMPIO.json")
test_d = pd.read_json("test_data_LIMPIO.json")

In [3]:
df = train_data
test_data = test_d

# Vader

Dado que VADER es un algoritmo basado en reglas y no un modelo entrenable, su precisión y desempeño pueden ser evaluados aplicándolo directamente sobre df

In [4]:
def apply_vader_and_label(dataframe):
    analyzer = SentimentIntensityAnalyzer()
    
    # Aplicar el análisis de sentimientos a cada texto
    dataframe[['neg', 'neu', 'pos', 'compound']] = dataframe['text'].apply(lambda x: pd.Series(analyzer.polarity_scores(x)))
    
    # Función para determinar la etiqueta basada en el puntaje compound
    def get_sentiment_label(compound_score):
        if compound_score >= 0.05:
            return 2  # positivo
        elif compound_score <= -0.05:
            return 0  # negativo
        else:
            return 1  # neutro
    
    # Aplicar la función a la columna compound para obtener las etiquetas predichas
    dataframe['predicted_label'] = dataframe['compound'].apply(get_sentiment_label)
    return dataframe

# Aplicar la función al DataFrame de entrenamiento
df = apply_vader_and_label(df)


# Mostrar los DataFrames actualizados
print("DataFrame de entrenamiento:")
print(df)


DataFrame de entrenamiento:
        label                                               text  \
0           0  love take seat appears though someone already ...   
1           1                                        fleet fleek   
2           0   nope flight leaving almost ca believe paid money   
3           2  awesome flight morning awesome crew even lande...   
4           2                          together best part flying   
...       ...                                                ...   
215080      2                              usual spielberg flair   
215081      1       shayamalan wanted tell story man loses faith   
215082      0  give poor dana carvey nothing really funny exp...   
215083      2  hugely rewarding experience every bit enlighte...   
215084      1                                       wallace film   

                             Dataset    neg    neu    pos  compound  \
0       Twitter US Airline Sentiment  0.381  0.387  0.232   -0.4019   
1       Twitt

# Metricas de rendimiento (informe clasificación)

VADER no es un modelo de aprendizaje supervisado que requiere un proceso de entrenamiento. Es un modelo de análisis de sentimientos basado en reglas y lexicón, lo que significa que no se entrena en un conjunto de datos antes de aplicarse a otros datos. Su funcionamiento se basa en un diccionario de palabras con puntajes predefinidos y reglas para calcular la polaridad del sentimiento.

In [5]:
# Generar el informe de clasificación para el conjunto de entrenamiento
train_report = classification_report(df['label'], df['predicted_label'], output_dict=True)
train_report_df = pd.DataFrame(train_report).transpose()
print("Informe de clasificación para el conjunto de entrenamiento")
print(train_report_df)

Informe de clasificación para el conjunto de entrenamiento
              precision    recall  f1-score        support
0              0.786524  0.379858  0.512298   96644.000000
1              0.410760  0.646706  0.502410   24498.000000
2              0.591220  0.817134  0.686057   93943.000000
accuracy       0.601241  0.601241  0.601241       0.601241
macro avg      0.596168  0.614566  0.566922  215085.000000
weighted avg   0.658421  0.601241  0.587065  215085.000000


: 


El uso de VADER para el análisis de sentimientos no se presta directamente a técnicas de validación cruzada (cross-validation) típicas del aprendizaje supervisado, ya que VADER es un algoritmo basado en reglas y lexicón, no un modelo que se entrena con datos específicos